In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install transformers

In [ ]:
import numpy as np
from gensim.models import FastText
import numpy as np
import torch
from transformers import BertModel, BertTokenizer

import pandas as pd
import numpy as np
import re, os, sys
from itertools import product

def read_nucleotide_sequences(file):
    if os.path.exists(file) == False:
        print('Error: file %s does not exist.' % file)
        sys.exit(1)
    with open(file) as f:
        records = f.read()
    if re.search('>', records) == None:
        print('Error: the input file %s seems not in FASTA format!' % file)
        sys.exit(1)
    records = records.split('>')[1:]
    fasta_sequences = []
    for fasta in records:
        array = fasta.split('\n')
        header, sequence = array[0].split()[0], re.sub('[^ACGTU-]', '-', ''.join(array[1:]).upper())
        header_array = header.split('|')
        name = header_array[0]
        label = header_array[1] if len(header_array) >= 2 else '0'
        label_train = header_array[2] if len(header_array) >= 3 else 'training'
        sequence = re.sub('U', 'T', sequence)
        fasta_sequences.append(sequence)
    return fasta_sequences

#!/usr/bin/env python
#_*_coding:utf-8_*_

import re

def check_fasta_with_equal_length(fastas):
    status = True
    lenList = set()
    for i in fastas:
        lenList.add(len(i[1]))
    if len(lenList) == 1:
        return True
    else:
        return False

def get_min_sequence_length(fastas):
    minLen = 10000
    for i in fastas:
        if minLen > len(i[1]):
            minLen = len(i[1])
    return minLen

def get_min_sequence_length_1(fastas):
    minLen = 10000
    for i in fastas:
        if minLen > len(re.sub('-', '', i[1])):
            minLen = len(re.sub('-', '', i[1]))
    return minLen
def readFasta(file):
    if os.path.exists(file) == False:
        print('Error: "' + file + '" does not exist.')
        sys.exit(1)

    with open(file) as f:
        records = f.read()

    if re.search('>', records) == None:
        print('The input file seems not in fasta format.')
        sys.exit(1)

    records = records.split('>')[1:]
    myFasta = []
    for fasta in records:
        array = fasta.split('\n')
        name, sequence = array[0].split()[0], re.sub('[^ARNDCQEGHILKMFPSTWYV-]', '-', ''.join(array[1:]).upper())
        myFasta.append([name, sequence])
    return myFasta


def extract_features(dna_sequences, vector_size=100, window=5, min_count=1):
    # Prepare data for FastText
    tokenized_sequences = [list(sequence) for sequence in dna_sequences]

    # Train FastText model
    model = FastText(tokenized_sequences, vector_size=vector_size, window=window, min_count=min_count)

    # Extract features
    features = np.zeros((len(dna_sequences), vector_size))
    for i, sequence in enumerate(tokenized_sequences):
        for token in sequence:
            features[i] += model.wv[token]
        features[i] /= len(sequence)

    return features

# Example usage
dna_sequences = ['ATCGATCGATCG', 'CGATCGATCGATCG']

fastas = read_nucleotide_sequences('/content/drive/MyDrive/Research Work/NLP-Features code/balanced_training_datasets/cd_ac4c_training_10.fasta')
features = extract_features(fastas)
data_csv=pd.DataFrame(features)
data_csv.to_csv('/content/drive/MyDrive/Research Work/NLP-Features code/balanced_training_datasets/FastText_feature_training_10.csv')

print(features)

In [9]:
!pip install fasttext

In [4]:
!pip install biopython



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.4 MB/s eta 0:00:00





In [3]:
import fasttext

# Path to your RNA sequence text file
data_path = "/content/drive/MyDrive/Research Work/NLP-Features code/Saweera_data/balanced_training_datasets/cd_ac4c_training_1.fasta"

# Train FastText model
model = fasttext.train_unsupervised(data_path, model='cbow', dim=100, epoch=10, minCount=1)

# Save the trained model
model.save_model("rna_fasttext_model.bin")

print("Model trained and saved successfully.")


Model trained and saved successfully.


In [8]:
import fasttext
from Bio import SeqIO
import numpy as np
import pandas as pd

# Load FastText model
# model = fasttext.load_model('/content/rna_fasttext_model.bin')  # Pre-trained FastText model, you can replace it with your own trained model
# Train FastText model
model = fasttext.train_unsupervised(input_file, model='cbow', dim=100, epoch=10, minCount=1)

# Function to convert RNA sequence into numerical features
def sequence_to_features(sequence, model):
    # Tokenize RNA sequence into k-mers (n-grams)
    k = 3  # You can adjust the k-mer size
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

    # Compute FastText embeddings for each k-mer
    embeddings = [model.get_word_vector(kmer) for kmer in kmers]

    # Calculate mean embedding as the feature representation
    if embeddings:
        feature = np.mean(embeddings, axis=0)
    else:
        feature = np.zeros(model.get_dimension())  # Return zero vector if no k-mers found

    return feature

# Function to read RNA sequences from FASTA file and convert to features
def fasta_to_features(input_file, model):
    sequences = []
    features = []

    for record in SeqIO.parse(input_file, "fasta"):
        sequences.append(str(record.seq))
        features.append(sequence_to_features(str(record.seq), model))

    return sequences, np.array(features)

# Read RNA sequences from FASTA file and convert to features
input_file = "/content/drive/MyDrive/Research Work/NLP-Features code/Saweera_data/balanced_training_datasets/cd_ac4c_training_1.fasta"
sequences, features = fasta_to_features(input_file, model)

# Create DataFrame for features
columns = [f"feature_{i+1}" for i in range(features.shape[1])]
df = pd.DataFrame(features, columns=columns)
df.insert(0, "Sequence", sequences)

# Save features as CSV
output_file = "rna_features.csv"
df.to_csv(output_file, index=False)

print("Features saved to", output_file)


Features saved to rna_features.csv


In [10]:
!pip install gensim 


In [11]:
import os
import re
import sys
import pandas as pd
import numpy as np
from gensim.models import FastText

def read_peptide_sequences(file):
    if not os.path.exists(file):
        print(f'Error: file {file} does not exist.')
        sys.exit(1)
    
    with open(file) as f:
        records = f.read()
    
    if '>' not in records:
        print(f'Error: the input file {file} seems not in FASTA format!')
        sys.exit(1)
    
    records = records.split('>')[1:]
    peptide_sequences = []
    for fasta in records:
        array = fasta.split('\n')
        header, sequence = array[0], ''.join(array[1:]).upper()
        peptide_sequences.append(sequence)
    
    return peptide_sequences

def extract_features(peptide_sequences, vector_size=100, window=5, min_count=1):
    # Prepare data for FastText
    tokenized_sequences = [list(sequence) for sequence in peptide_sequences]

    # Train FastText model
    model = FastText(tokenized_sequences, vector_size=vector_size, window=window, min_count=min_count)

    # Extract features
    features = np.zeros((len(peptide_sequences), vector_size))
    for i, sequence in enumerate(tokenized_sequences):
        for token in sequence:
            features[i] += model.wv[token]
        features[i] /= len(sequence)

    return features

def main():
    # File paths
    input_file = '/kaggle/input/ace-dataset/ACE_full_dataset.txt'
    output_file = '/kaggle/working/fasttext_features_ACE.csv'
    
    # Read peptide sequences
    peptide_sequences = read_peptide_sequences(input_file)
    
    # Extract features using FastText
    features = extract_features(peptide_sequences)
    
    # Save features to CSV
    features_df = pd.DataFrame(features)
    features_df.to_csv(output_file, index=False)
    
    print("Features extracted and saved to CSV successfully.")
    print(features)

if __name__ == "__main__":
    main()


Features extracted and saved to CSV successfully.
[[-0.0782537   0.04756918  0.04022702 ...  0.09361638  0.01984505
   0.01214317]
 [-0.07814427  0.04698247  0.04004453 ...  0.09273018  0.01868749
   0.01251137]
 [-0.07606108  0.04613785  0.0391121  ...  0.08946893  0.01659791
   0.01310674]
 ...
 [-0.09876951  0.06239232  0.04844261 ...  0.11263584  0.0259387
   0.01808453]
 [-0.09526661  0.05825514  0.04923445 ...  0.11082024  0.02411642
   0.01881375]
 [-0.09460388  0.06089057  0.04749231 ...  0.11063424  0.0245214
   0.01726293]]


Word 2 vectore feature using CWOB

In [13]:
import os
import re
import sys
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from collections import Counter


def read_peptide_sequences(file):
    if not os.path.exists(file):
        print(f'Error: file {file} does not exist.')
        sys.exit(1)
    
    with open(file) as f:
        records = f.read()
    
    if '>' not in records:
        print(f'Error: the input file {file} seems not in FASTA format!')
        sys.exit(1)
    
    records = records.split('>')[1:]
    peptide_sequences = []
    for fasta in records:
        array = fasta.split('\n')
        header, sequence = array[0], ''.join(array[1:]).upper()
        peptide_sequences.append(sequence)
    
    return peptide_sequences

def extract_features(peptide_sequences, vector_size=100, window=5, min_count=1):
    # Prepare data for Word2Vec
    tokenized_sequences = [list(sequence) for sequence in peptide_sequences]

    # Train Word2Vec model
    model = Word2Vec(tokenized_sequences, vector_size=vector_size, window=window, min_count=min_count)
    
    # Create a vocabulary list
    vocabulary = list(model.wv.index_to_key)

    # Extract BoW + Word2Vec features
    features = []
    for sequence in tokenized_sequences:
        # Bag of Words representation
        bow = Counter(sequence)
        bow_vector = [bow[token] for token in vocabulary]
        
        # Word2Vec representation
        word2vec_vector = np.zeros(vector_size)
        for token in sequence:
            if token in model.wv:
                word2vec_vector += model.wv[token]
        word2vec_vector /= len(sequence)
        
        # Combine BoW and Word2Vec vectors
        combined_vector = np.concatenate([bow_vector, word2vec_vector])
        features.append(combined_vector)
    
    return np.array(features), vocabulary

def main():
    # File paths
    input_file = '/kaggle/input/ace-dataset/ACE_full_dataset.txt'
    output_file = '/kaggle/working/bow_word2vec_features_ACE.csv'
    
    # Read peptide sequences
    peptide_sequences = read_peptide_sequences(input_file)
    
    # Extract features using BoW + Word2Vec
    features, vocabulary = extract_features(peptide_sequences)
    
    # Create a DataFrame with the combined features
    bow_columns = [f'bow_{token}' for token in vocabulary]
    word2vec_columns = [f'word2vec_{i}' for i in range(features.shape[1] - len(vocabulary))]
    columns = bow_columns + word2vec_columns
    
    features_df = pd.DataFrame(features, columns=columns)
    features_df.to_csv(output_file, index=False)
    
    print("Features extracted and saved to CSV successfully.")
    print(features)

if __name__ == "__main__":
    main()


Features extracted and saved to CSV successfully.
[[ 0.          1.          0.         ... -0.01750997  0.06654619
   0.0491765 ]
 [ 0.          1.          0.         ... -0.01652273  0.06486657
   0.04985909]
 [ 0.          1.          0.         ... -0.01658617  0.06147339
   0.05035048]
 ...
 [ 1.          0.          0.         ... -0.02397426  0.08200689
   0.06096922]
 [ 1.          1.          0.         ... -0.02460871  0.0787589
   0.06186177]
 [ 1.          1.          0.         ... -0.02208124  0.0797771
   0.05769714]]
